In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from librosa import core, onset, feature, display
import soundfile as sf
import umap
from IPython.display import Audio
import sklearn

In [2]:
df = pd.read_csv("./songs/predict/predict.csv")
df.head()

,file_id,genus,species
0,1,Unknown,species


In [3]:
def load_audio(file_id):
    data, samplerate = sf.read("./songs/predict/xc"+str(file_id)+".flac")
    s = len(data)/samplerate
    sg = feature.melspectrogram(data, sr=samplerate, hop_length=512)
    
    # Take mean amplitude M from frame with highest energy
    centerpoint = np.argmax(sg.mean(axis=0))
    M = sg[:,centerpoint].mean()
    
    # Filter out all frames with energy less than 5% of M
    mask = sg.mean(axis=0)>=M/20

    audio_mask = np.zeros(len(data), dtype=bool)
    for i in range(0,len(mask)):
        audio_mask[i*512:] = mask[i]
    return sg, mask, data, audio_mask, samplerate

In [4]:
df['length'] = np.zeros(len(df))

waves = {}


for file_id in df['file_id']:
    sg, mask, data, audio_mask, sample_rate = load_audio(file_id)
    waves[file_id] = data[audio_mask]
    df.loc[df['file_id'] == file_id,'length'] = len(data[audio_mask])

In [5]:
df['windows'] = df['length'].apply(lambda x: int(x/6.144000e+03))
df

,file_id,genus,species,length,windows
0,1,Unknown,species,116736.0,19


In [6]:
n_windows = df.groupby('species')['windows'].sum().min()
n_windows

19

In [7]:
windows = {}

for file_id in df['file_id']:
    wave = waves[file_id]
    species = df[df['file_id']==file_id]['genus'].values[0] + "_" + df[df['file_id']==file_id]['species'].values[0]
    if species not in windows:
        windows[species] = []
    for i in range(0, int(len(wave)/6.144000e+03)):
        windows[species].append(wave[i:int(i+6.144000e+03)])

In [8]:
windows_fixed = {}

for species in windows.keys():
    windows_fixed[species] = []
    ws = windows[species]
    index = np.random.choice(len(ws), n_windows, replace=False)
    for i in range(0, len(ws)):
        if i in index:
            windows_fixed[species].append(ws[i])

In [9]:
new_dataset = pd.DataFrame()

for species in windows_fixed.keys():
    for i in range(0,n_windows):
        data_point = {'species':species.split('_')[1], 'genus':species.split('_')[0]}
        spec_centroid = feature.spectral_centroid(windows_fixed[species][i])[0]
        chroma = feature.chroma_stft(windows_fixed[species][i], sample_rate)
        for j in range(0,13):
            data_point['spec_centr_'+str(j)] = spec_centroid[j]
            for k in range(0,12):
                data_point['chromogram_'+str(k)+"_"+str(j)] = chroma[k,j]
        new_dataset = new_dataset.append(data_point,ignore_index=True)

new_dataset.head()

C:\Users\KRISHNA PRASAD P\AppData\Local\Programs\Python\Python37\lib\site-packages\librosa\core\pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


,chromogram_0_0,chromogram_0_1,chromogram_0_10,chromogram_0_11,chromogram_0_12,chromogram_0_2,chromogram_0_3,chromogram_0_4,chromogram_0_5,chromogram_0_6,...,spec_centr_12,spec_centr_2,spec_centr_3,spec_centr_4,spec_centr_5,spec_centr_6,spec_centr_7,spec_centr_8,spec_centr_9,species
0,0.001547,0.000367,0.000011,0.000365,0.002190,0.000009,0.000011,0.000015,0.000092,0.016634,...,3135.864570,2992.033112,2949.803712,2929.013309,2789.240893,2623.994022,2707.139473,3261.524923,3249.137130,species
1,0.000296,0.000083,0.000011,0.000484,0.002906,0.000009,0.000011,0.000015,0.000094,0.016739,...,3131.321942,2991.931160,2949.730514,2928.929492,2788.884704,2623.710382,2708.032966,3262.092133,3248.791819,species
2,0.000343,0.000093,0.000011,0.000124,0.000673,0.000009,0.000011,0.000015,0.000095,0.016845,...,3078.880478,2991.829314,2949.657408,2928.844811,2788.528609,2623.426835,2708.930140,3262.655859,3248.446183,species
3,0.001569,0.000367,0.000011,0.000054,0.000279,0.000009,0.000011,0.000015,0.000097,0.016951,...,3070.662987,2991.727550,2949.584339,2928.759300,2788.172592,2623.143393,2709.830956,3263.216117,3248.100236,species
4,0.000857,0.000213,0.000011,0.000399,0.002418,0.000009,0.000011,0.000015,0.000098,0.017059,...,3119.158780,2991.625859,2949.511301,2928.672991,2787.816621,2622.860063,2710.735346,3263.772931,3247.753993,species


In [10]:
features= list(new_dataset.columns)
features.remove('species')
features.remove('genus')

X = new_dataset[features].values

In [11]:
loaded_model = pickle.load(open('svm.sav', 'rb'))

In [12]:
ans=loaded_model.predict(X)
dic={}
s=set(ans)
for i in s:
    dic[i]=sum([1 for j in list(ans) if i==j])
dic['jadf']=12
sol=None
res=None
for i in dic.items():
    if sol is None or sol<i[1]:
        sol=i[1]
        res=i[0]
res

'familiaris'

In [13]:
loaded_model = pickle.load(open('naive.sav', 'rb'))

In [14]:
ans=loaded_model.predict(X)
dic={}
s=set(ans)
for i in s:
    dic[i]=sum([1 for j in list(ans) if i==j])
dic['jadf']=12
sol=None
res=None
for i in dic.items():
    if sol is None or sol<i[1]:
        sol=i[1]
        res=i[0]
res

'trochilus'